- [Reference kernel](https://www.kaggle.com/willkoehrsen/automated-feature-engineering-basics)

In [ ]:
# install featuretools
# !pip install featuretools

In [3]:
# pandas and numpy for data manipulation
import pandas as pd
import numpy as np

# featuretools for automated feature engineering
import featuretools as ft

# matplotlit and seaborn for visualizations
import matplotlib.pyplot as plt
plt.rcParams['font.size'] = 22
import seaborn as sns

# Suppress warnings from pandas
import warnings
warnings.filterwarnings('ignore')

Problem
The Home Credit Default Risk competition is a supervised classification machine learning task. The objective is to use historical financial and socioeconomic data to predict whether or not an applicant will be able to repay a loan. This is a standard supervised classification task:

Supervised: The labels are included in the training data and the goal is to train a model to learn to predict the labels from the features
Classification: The label is a binary variable, 0 (will repay loan on time), 1 (will have difficulty repaying loan)
Dataset
The data is provided by Home Credit, a service dedicated to provided lines of credit (loans) to the unbanked population.

There are 7 different data files:

- application_train/application_test: the main training and testing data with information about each loan application at Home Credit. Every loan has its own row and is identified by the SK_ID_CURR. The training application data comes with the TARGET with indicating 0: the loan was repaid and 1: the loan was not repaid.
- bureau: data concerning client's previous credits from other financial institutions. Each previous credit has its own row in bureau and is identified by the SK_ID_BUREAU, Each loan in the application data can have multiple previous credits.
- bureau_balance: monthly data about the previous credits in bureau. Each row is one month of a previous credit, and a single previous credit can have multiple rows, one for each month of the credit length.
- previous_application: previous applications for loans at Home Credit of clients who have loans in the application data. Each current loan in the application data can have multiple previous loans. Each previous application has one row and is identified by the feature SK_ID_PREV.
- POS_CASH_BALANCE: monthly data about previous point of sale or cash loans clients have had with Home Credit. Each row is one month of a previous point of sale or cash loan, and a single previous loan can have many rows.
- credit_card_balance: monthly data about previous credit cards clients have had with Home Credit. Each row is one month of a credit card balance, and a single credit card can have many rows.
- installments_payment: payment history for previous loans at Home Credit. There is one row for every made payment and one row for every missed payment

In [16]:
##############################################################
##################  import data  #############################
##############################################################

# Read in the datasets and limit to the first 1000 rows (sorted by SK_ID_CURR) 
# This allows us to actually see the results in a reasonable amount of time! 
app_train = pd.read_csv('./input/home-credit-default-risk/application_train.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
app_test = pd.read_csv('./input/home-credit-default-risk/application_test.csv').sort_values('SK_ID_CURR').reset_index(drop = True).loc[:1000, :]
bureau = pd.read_csv('./input/home-credit-default-risk/bureau.csv').sort_values(['SK_ID_CURR', 'SK_ID_BUREAU']).reset_index(drop = True).loc[:1000, :]
bureau_balance = pd.read_csv('./input/home-credit-default-risk/bureau_balance.csv').sort_values('SK_ID_BUREAU').reset_index(drop = True).loc[:1000, :]
cash = pd.read_csv('./input/home-credit-default-risk/POS_CASH_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
credit = pd.read_csv('./input/home-credit-default-risk/credit_card_balance.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
previous = pd.read_csv('./input/home-credit-default-risk/previous_application.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]
installments = pd.read_csv('./input/home-credit-default-risk/installments_payments.csv').sort_values(['SK_ID_CURR', 'SK_ID_PREV']).reset_index(drop = True).loc[:1000, :]

In [17]:
# add identity column
app_train['set'] = 'train'
app_test['set'] = 'test'
app_test['TARGET'] = np.nan

# append the data frames
app = app_train.append(app_test, ignore_index = True)

## Featuretool basics

[Featuretools](https://docs.featuretools.com/index.html) is an open-source Python library for automatically creating features out of a set of related tables using a technique called [deep feature synthesis](http://www.jmaxkanter.com/static/papers/DSAA_DSM_2015.pdf). Automated feature engineering, like many topics in machine learning, is a complex subject built upon a foundation of simpler ideas. By going through these ideas one at a time, we can build up our understanding of how featuretools which will later allow for us to get the most out of it.

There are a few concepts that we will cover along the way:

- Entities and EntitySets
- Relationships between tables
- Feature primitives: aggregations and transformations
- Deep feature synthesis

### Entities and Entitysets
An **entity** is simply a table or in Pandas, a dataframe. The observations are in the rows and the features in the columns

An **EntitySet** is a collection of tables and the relationships between them. This can be thought of a data structute with its own methods and attributes. Using an EntitySet allows us to group together multiple tables and manipulate them much quicker than individual tables.

First we'll make an empty entityset named clients to keep track of all the data.

In [18]:
# Entity set with id applications
es = ft.EntitySet(id = 'clients')

Now we define each entity, or table of data. We need to pass in an index if the data has one or make_index = True if not. Featuretools will automatically infer the types of variables, but we can also change them if needed. For intstance, if we have a categorical variable that is represented as an integer we might want to let featuretools know the right type.

In [19]:
# Entities with unique index
es = es.entity_from_dataframe(entity_id='app',dataframe=app,index='SK_ID_CURR')
es = es.entity_from_dataframe(entity_id = 'bureau', dataframe = bureau, index = 'SK_ID_BUREAU')
es = es.entity_from_dataframe(entity_id = 'previous', dataframe = previous, index = 'SK_ID_PREV')

# Entities that do not have unique index
es = es.entity_from_dataframe(entity_id='bureau_balance',dataframe=bureau_balance,make_index=True,index='bureaubalance_index')
es = es.entity_from_dataframe(entity_id = 'cash', dataframe = cash, 
                              make_index = True, index = 'cash_index')
es = es.entity_from_dataframe(entity_id = 'installments', dataframe = installments,
                              make_index = True, index = 'installments_index')
es = es.entity_from_dataframe(entity_id = 'credit', dataframe = credit,
                              make_index = True, index = 'credit_index')

### Relationships

Parent > Child > Grandchild

In [20]:
# Rellationship between app and bureau
r_app_bureau = ft.Relationship(es['app']['SK_ID_CURR'], es['bureau']['SK_ID_CURR'])

# Relationship between bureau and bureau balance
r_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], es['bureau_balance']['SK_ID_BUREAU'])

# Relationship between current app and previous apps
r_app_previous = ft.Relationship(es['app']['SK_ID_CURR'], es['previous']['SK_ID_CURR'])

# Relationships between previous apps and cash, installments, and credit
r_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], es['cash']['SK_ID_PREV'])
r_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], es['installments']['SK_ID_PREV'])
r_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], es['credit']['SK_ID_PREV'])

In [23]:
# Add in the defined relationships to entity set
es = es.add_relationships([r_app_bureau, r_bureau_balance, r_app_previous,
                           r_previous_cash, r_previous_installments, r_previous_credit])
# print out the entity set
es


2019-04-07 08:34:04,984 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: bureau.SK_ID_CURR -> app.SK_ID_CURR>
2019-04-07 08:34:04,989 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU>
2019-04-07 08:34:04,989 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: previous.SK_ID_CURR -> app.SK_ID_CURR>
2019-04-07 08:34:04,990 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: cash.SK_ID_PREV -> previous.SK_ID_PREV>
2019-04-07 08:34:04,991 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: installments.SK_ID_PREV -> previous.SK_ID_PREV>
2019-04-07 08:34:04,992 featuretools.entityset - WARNING    Not adding duplicate relationship: <Relationship: credit.SK_ID_PREV -> previous.SK_ID_PREV>


Entityset: clients
  Entities:
    app [Rows: 2002, Columns: 123]
    bureau [Rows: 1001, Columns: 17]
    previous [Rows: 1001, Columns: 37]
    bureau_balance [Rows: 1001, Columns: 4]
    cash [Rows: 1001, Columns: 9]
    installments [Rows: 1001, Columns: 9]
    credit [Rows: 1001, Columns: 24]
  Relationships:
    bureau.SK_ID_CURR -> app.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

We need to be careful to not create a diamond graph where there are multiple paths from a parent to a child

```![image.png](attachment:image.png)```

### Feature Primitives

A feature primitive is an operation applied to a table or a set of tables to create a feature. These represent simple calculations, many of which we already use in manual feature engineering, that can be stacked on top of each other to create complex features. Feature primitives fall into two categories:

- **Aggregation:** function that groups together child datapoints for each parent and then calculates a statistic such as mean, min, max, or standard deviation. An example is calculating the maximum previous loan amount for each client. An aggregation works across multiple tables using relationships between tables
- **Transformation:** an operation applied to one or more columns in a single table. An example would be taking the absolute value of a column, or finding the difference between two columns in one table

A list of the available features primitives in featuretools can be viewed below.

In [53]:
# list of primitives

primitives = ft.list_primitives()
pd.options.display.max_colwidth == 100
primitives[primitives['type'] == 'aggregation'].head()

,name,type,description
0,min,aggregation,Finds the minimum non-null value of a numeric ...
1,max,aggregation,Finds the maximum non-null value of a numeric ...
2,median,aggregation,Finds the median value of any feature with wel...
3,mode,aggregation,Finds the most common element in a categorical...
4,any,aggregation,Test if any value is 'True'.


In [70]:
primitives[primitives['type'] == 'transform'].head()

,name,type,description
20,greater_than,transform,
21,year,transform,Transform a Datetime feature into the year.
22,add_numeric,transform,
23,less_than_scalar,transform,
24,days_since,transform,"For each value of the base feature, compute th..."


In [55]:
primitives['type'].value_counts()

transform      56
aggregation    20
Name: type, dtype: int64

### Deep Feature Synthesis

Deep Feature Synthesis (DFS) is the process featuretools uses to make new features. DFS stacks feature primitives to form features with a "depth" equal to the number of primitives. For example, if we take the maximum value of a client's previous loans (say MAX(previous.loan_amount)), that is a "deep feature" with a depth of 1. To create a feature with a depth of two, we could stack primitives by taking the maximum value of a client's average montly payments per previous loan (such as MAX(previous(MEAN(installments.payment)))). The original paper on automated feature engineering using deep feature synthesis is worth a read.

To perform DFS in featuretools, we use the dfs function passing it an entityset, the target_entity (where we want to make the features), the agg_primitives to use, the trans_primitives to use and the max_depth of the features. Here we will use the default aggregation and transformation primitives, a max depth of 2, and calculate primitives for the app entity. Because this process is computationally expensive, we can run the function using features_only = True to return only a list of the features and not calculate the features themselves. This can be useful to look at the resulting features before starting an extended computation.

In [71]:
# Default primitives from featuretools
default_agg_primitives =  ["sum", "std", "max", "skew", "min", "mean", "count", "percent_true", "num_unique", "mode"]
default_trans_primitives =  ["year", "day","month","weekday","haversine","num_words","num_characters"]

# DFS with specified primitives
feature_names = ft.dfs(entityset = es, target_entity = 'app',
                       trans_primitives = default_trans_primitives,
                       agg_primitives=default_agg_primitives, 
                       max_depth = 2, features_only=True)

print('%d Total Features' % len(feature_names))

1697 Total Features


In [72]:
# DFS with default primitives
feature_matrix, feature_names = ft.dfs(entityset = es, target_entity = 'app',
                                       trans_primitives = default_trans_primitives,
                                       agg_primitives=default_agg_primitives, 
                                        max_depth = 2, features_only=False, verbose = True)

pd.options.display.max_columns = 1700
feature_matrix.head(10)

Built 1697 features
Elapsed: 01:09 | Remaining: 00:00 | Progress: 100%|██| Calculated: 11/11 chunks


AMT_ANNUITY  AMT_CREDIT  AMT_GOODS_PRICE  AMT_INCOME_TOTAL  \
SK_ID_CURR                                                               
100001          20560.5    568800.0         450000.0          135000.0   
100002          24700.5    406597.5         351000.0          202500.0   
100003          35698.5   1293502.5        1129500.0          270000.0   
100004           6750.0    135000.0         135000.0           67500.0   
100005          17370.0    222768.0         180000.0           99000.0   
100006          29686.5    312682.5         297000.0          135000.0   
100007          21865.5    513000.0         513000.0          121500.0   
100008          27517.5    490495.5         454500.0           99000.0   
100009          41301.0   1560726.0        1395000.0          171000.0   
100010          42075.0   1530000.0        1530000.0          360000.0   

            AMT_REQ_CREDIT_BUREAU_DAY  AMT_REQ_CREDIT_BUREAU_HOUR  \
SK_ID_CURR                                                          
100001                            0.0                         0.0   
100002                            0.0                         0.0   
100003                            0.0                         0.0   
100004                            0.0                         0.0   
100005                            0.0                         0.0   
100006                            NaN                         NaN   
100007                            0.0                         0.0   
100008                            0.0                         0.0   
100009                            0.0                         0.0   
100010                            0.0                         0.0   

            AMT_REQ_CREDIT_BUREAU_MON  AMT_REQ_CREDIT_BUREAU_QRT  \
SK_ID_CURR                                                         
100001                            0.0                        0.0   
100002                            0.0                        0.0   
100003                            0.0                        0.0   
100004                            0.0                        0.0   
100005                            0.0                        0.0   
100006                            NaN                        NaN   
100007                            0.0                        0.0   
100008                            0.0                        1.0   
100009                            1.0                        1.0   
100010                            0.0                        0.0   

            AMT_REQ_CREDIT_BUREAU_WEEK  AMT_REQ_CREDIT_BUREAU_YEAR  \
SK_ID_CURR                                                           
100001                             0.0                         0.0   
100002                             0.0                         1.0   
100003                             0.0                         0.0   
100004                             0.0                         0.0   
100005                             0.0                         3.0   
100006                             NaN                         NaN   
100007                             0.0                         0.0   
100008                             0.0                         1.0   
100009                             0.0                         2.0   
100010                             0.0                         0.0   

            APARTMENTS_AVG  APARTMENTS_MEDI  APARTMENTS_MODE  \
SK_ID_CURR                                                     
100001              0.0660           0.0666           0.0672   
100002              0.0247           0.0250           0.0252   
100003              0.0959           0.0968           0.0924   
100004                 NaN              NaN              NaN   
100005                 NaN              NaN              NaN   
100006                 NaN              NaN              NaN   
100007                 NaN              NaN              NaN   
100008                 NaN              NaN              NaN

In [73]:
feature_matrix.shape

(2002, 1697)